# Anomaly Detection With The TraceLog API

In [ ]:
#r "nuget:Microsoft.Diagnostics.Tracing.TraceEvent"
#r "nuget:XPlot.Plotly"
#r "nuget:XPlot.GoogleCharts"
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.TimeSeries"

open Microsoft.Diagnostics.Tracing;
open Microsoft.Diagnostics.Tracing.Analysis;
open Microsoft.Diagnostics.Tracing.Etlx;
open Microsoft.Diagnostics.Tracing.Session;
open Microsoft.Diagnostics.Tracing.Parsers.Clr;
open Microsoft.ML
open Microsoft.ML.Data
open Microsoft.ML.Transforms.TimeSeries
open XPlot.Plotly
open XPlot.GoogleCharts
open System.Collections.Generic

Installed Packages Microsoft.Diagnostics.Tracing.TraceEvent, 2.0.74 Microsoft.ML, 1.7.0 Microsoft.ML.TimeSeries, 1.7.0 XPlot.GoogleCharts, 3.0.1 XPlot.Plotly, 4.0.6

In [ ]:
let ETL_FILEPATH = @"C:\Users\mukun\OneDrive\Documents\CallstackShmuff.etl\CallstackShmuff.etl" 

let session = new TraceEventSession("TestSession", ETL_FILEPATH)

let traceLog = TraceLog.OpenOrConvert(ETL_FILEPATH)

In [ ]:
type Input() =
    [<DefaultValue>]
    [<LoadColumn(0)>]
    val mutable public timestamp : double 

    [<DefaultValue>]
    [<LoadColumn(1)>]
    val mutable public value : float32

type Prediction() = 
    [<DefaultValue>]
    [<VectorType(3)>]
    val mutable public Prediction : double[]

In [ ]:
let allocationAmountForDevenv =
    traceLog.Events
    |> Seq.filter(fun e -> e.ProcessName = "devenv" && e.EventName.Contains("GC/AllocationTick"))
    |> Seq.take 100
    |> Seq.map(fun e -> Input(timestamp = e.TimeStampRelativeMSec, value = float32 (e.PayloadByName("AllocationAmount").ToString())))

let ctx = MLContext()

let dataView = 
  ctx
    .Data
    .LoadFromEnumerable<Input>(allocationAmountForDevenv)
      
let anomalyPValueHistoryLength = 30
let changePointPValueHistoryLength = 10
let anomalyConfidence = 95
let changePointConfidence = 95

let anomalyPipeline =
    ctx.Transforms.DetectIidSpike(
      outputColumnName = "Prediction",
      inputColumnName = "value",
      side = AnomalySide.TwoSided,
      confidence = anomalyConfidence, 
      pvalueHistoryLength = anomalyPValueHistoryLength)

let trainedAnomalyModel = anomalyPipeline.Fit(ctx.Data.LoadFromEnumerable(List<Input>()))
let transformedAnomalyData = trainedAnomalyModel.Transform(dataView)

let anomalies = 
    ctx.Data.CreateEnumerable<Prediction>(transformedAnomalyData, reuseRowObject = false)

let allocRate = 
    allocationAmountForDevenv
    |> Seq.map(fun i -> double(i.timestamp), double(i.value))
    |> Seq.toList

let anomalyChartData = 
  anomalies
  |> Seq.mapi(fun i p -> double(p.Prediction.[0]), double(p.Prediction.[1]), fst (allocRate.[i]))
  |> Seq.toList

let anomalyToChart = 
    anomalyChartData
    |> List.filter(fun (x, b, c) -> x = 1.0)
    |> List.map(fun (x, b, c) -> (c, b))

display(anomalyToChart)

[allocRate; anomalyToChart]
|> Chart.Combo
|> Chart.Show

index,Item1,Item2
0,1516.0816,108784
1,1516.652,105680
2,1710.3076,115224
3,2814.5199,115072
4,2818.3139,113992


![Anomalies With Allocations]("./Images/AnomaliesWithAllocations.png")